In [1]:
import shutil
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pwd = os.getcwd()
#pwd += "/rheo_bench/channel/"
pwd

'/home/etpaler/test/RHEO_benchmarks/surf_normal/circular_surface'

In [3]:
#parameter grids
var_sf_grid = {"0.05":"20","0.1":"10","0.2":"5","0.25":"4","0.5":"2"}
#var_sf_grid = {"0.1":"10"}
var_kernel_grid = {"Quintic":"5"}


# Generate LAMMPS Input Scripts

In [4]:
#Generate lammps files
filenames = []
for sf in var_sf_grid:
    for kernel in var_kernel_grid:
        run_code = "0_"+var_kernel_grid[kernel]+"_"+var_sf_grid[sf] #0 denotes circular surface
        filename = "in."+run_code+".lmp"
        file = shutil.copyfile("lammps_in_circle.template",pwd+"/"+filename)
        file_in = open(file,"rt")
        file_text = file_in.read()

        #Replace variables
        file_text = file_text.replace("var_sf",sf)
        file_text = file_text.replace("var_kernel",kernel)
        file_text = file_text.replace("var_data","circle_surface_"+sf+".data")
        file_text = file_text.replace("var_filename",run_code)

        file_out = open(file,"wt")
        file_out.write(file_text)
        file_out.close()
        
        filenames.append(filename)

# Write the Jobscript

### Solo

In [10]:
account="FY150028"
job_name="TestChannel"
time="24:0:0"
partition="batch"
nodes="1"
ntasks_per_node="16"
run_command = "mpirun -np 16 /ascldap/users/etpaler/dev/lammps_rheo/lammps/build/lmp -in "

In [11]:
with open("jobscript","w") as jobscript:
        #---shebang---#
        jobscript.write("#!/bin/bash\n\n")

        #---slurm commands---#
        jobscript.write("#SBATCH --account={}\n".format(account))
        jobscript.write("#SBATCH --job-name={}\n".format(job_name))
        jobscript.write("#SBATCH --time={}\n".format(time))
        jobscript.write("#SBATCH --partition={}\n".format(partition))
        jobscript.write("#SBATCH --nodes={}\n".format(nodes))
        jobscript.write("#SBATCH --ntasks-per-node={}\n\n".format(ntasks_per_node))
        
        jobscript.write("module load cmake/3.20.3 intel/18.0.0.128 openmpi-intel/4.1 mkl/18.0.0.128\n\n")
        
        for filename in filenames:
            jobscript.write(run_command+filename+"\n")
            
        jobscript.write("\n")

### WSL

In [34]:
run_command = "mpirun -np 1 ~/dev/rlammps_inlet/build/lmp -in "

with open("jobscript","w") as jobscript:
        jobscript.write("#!/bin/bash\n\n")

        for filename in filenames:
            jobscript.write(run_command+filename+"\n")
            
        jobscript.write("\n")